In [1]:
import os
import pandas as pd
import config_data
import datetime
from unidecode import unidecode
from urllib.parse import unquote

TARGET_LISTINGS_DIR = config_data.AIRBNB_DATA_DIR

In [2]:
def proc_dirto_row(loc_dir):
    #read all CSVs in loc_dir and retrieve the number of unique listings in each CSV
    timewise_dict = {}
    min_year =2020
    min_month = 12
    max_year = 0
    max_month = 1
    for csvfile in os.listdir(loc_dir):
        csvfile_fullpath = os.path.join(loc_dir,csvfile)
        if csvfile.split('.')[-1] == "csv":
            try:
                df = pd.read_csv(csvfile_fullpath)
                csvfile_no_ext = csvfile[:csvfile.rfind('.')]
                year, month, day = tuple(csvfile_no_ext.split('-')[-3:])
                year = int(year)
                month = int(month)
                if year <= min_year:
                    min_year = year
                    if month < min_month:
                        min_month = month
                if year >= max_year:
                    max_year = year
                    if month > max_month:
                        max_month = month
                k = (year, month, 1)
                timewise_dict[k]= len(pd.unique(df['id']))
#                 print(f"File Date:{(month,year)} Number of unique listings:{len(pd.unique(df['id']))}")
            except OSError:
                print(f"Skipping {csvfile_fullpath}")
    sorted_by_time = [(k,timewise_dict[k]) for k in sorted(timewise_dict.keys())]
    sorted_by_time = sorted_by_time[::-1]
    return sorted_by_time

In [3]:
def proc_dirto_df(loc_dir):
    #generate a df for the location with date, listings
    sorted_by_time = proc_dirto_row(loc_dir)
    proc_date_list = []
    proc_listing_list = []
    prev_month=0
    prev_year = 0
    year = 0
    month = 0
    prev_n_listings = 0
    curr_n_listings = 0
    for i in range(len(sorted_by_time)):
        prev_month = month
        prev_year = year
        prev_n_listings = curr_n_listings
        year=sorted_by_time[i][0][0]
        month=sorted_by_time[i][0][1]
        day = sorted_by_time[i][0][2]
        curr_n_listings = sorted_by_time[i][1]
        if not month == (prev_month-1) and prev_year == year:
            #interpolation of n_listings for missing months
            for j in range(month+1,prev_month):
                interp_month = j
                interp_date = str(year) + '-' + str(j) +'-'+str(day)
                prev_wt = (j - month) / (prev_month - month)
                curr_wt = 1 - prev_wt
                interp_listings = int(prev_wt * prev_n_listings + curr_wt * curr_n_listings)
                proc_date_list.append((interp_date, interp_listings))
                
        date = str(year) + '-' + str(month) +'-'+str(day)
        proc_date_list.append((date,curr_n_listings))
    df = pd.DataFrame(proc_date_list, columns = ['date','n_listings'])
    df = df.set_index('date')
    return df

In [6]:
cities = []
for k, v in config_data.US_state_to_city.items():
    for t in v:
        cities.append(t)
for k,v in config_data.country_to_city.items():
    cities.append(v)
for v in config_data.misc_cities:
    cities.append(v)
print(cities)

['Asheville', 'Austin', 'Boston', 'Cambridge', 'Broward County', 'Chicago', 'Clark County', 'Columbus', 'Denver', 'Hawaii', 'Jersey City', 'Los Angeles', 'Oakland', 'Pacific Grove', 'San Diego', 'San Francisco', 'San Mateo County', 'Santa Clara County', 'Santa Cruz County', 'Nashville', 'New Orleans', 'New York City', 'Portland', 'Salem', 'Rhode Island', 'Seattle', 'Twin Cities MSA', 'Washington D.C.', 'Munich', 'Madrid', 'Athens', 'Stockholm', 'Edinburgh', 'Shanghai', 'Singapore', 'Tokyo', 'Cape Town', 'Sydney', 'Ottawa', 'Washington D.C.', 'Mexico City', 'Rio de Janeiro', 'Buenos Aires', 'Beijing']


In [8]:
RAW_DATA_DIR = os.path.join('./Airbnb_Listings_Gz')

for loc_dir in os.listdir(RAW_DATA_DIR):
    loc_dir_fullpath = os.path.join(RAW_DATA_DIR, loc_dir)
#     print(loc_dir_fullpath)
    mergedname = unidecode(unquote(loc_dir.replace("-", "")))
    for city in cities:
        citylower_nospace = city.lower().replace(" ","")
        if citylower_nospace in mergedname:
#             print(f"getting df for {city}")
            citydf = proc_dirto_df(loc_dir_fullpath)
            df = citydf
            #save the df
            citydir = os.path.join(TARGET_LISTINGS_DIR,city)
            if not os.path.exists(citydir):
                os.makedirs(citydir)
            df.to_csv(os.path.join(citydir,'listings.csv'))
            print(f"saved for {city}")

saved for Ottawa
saved for Cambridge
saved for Athens
saved for Beijing
saved for Austin
saved for Santa Clara County
saved for Jersey City
saved for Los Angeles
saved for Mexico City
saved for Nashville
saved for Seattle
saved for Singapore
saved for San Mateo County
saved for New York City
saved for Hawaii
saved for Tokyo
saved for Broward County
saved for Twin Cities MSA
saved for San Diego
saved for San Francisco
saved for Santa Cruz County
saved for Rhode Island
saved for Boston
saved for Chicago
saved for Clark County
saved for Sydney
saved for Rio de Janeiro
saved for Portland
saved for Madrid
saved for Denver
saved for Asheville
saved for Oakland
saved for Columbus
saved for Buenos Aires
saved for Cape Town
saved for New Orleans
saved for Pacific Grove
saved for Munich
saved for Shanghai
saved for Stockholm
saved for Salem
saved for Edinburgh


In [ ]:
df = pd.read_csv

In [ ]:
import airbnb_dataset

myABData = airbnb_dataset.AirbnbDataset()